-------------- Real Project Time -------------

Use tensorflow.keras library for models. And you are free to use chatgpt here at this stage - Yayyyyyyyy\
Use it where you need help, not to complete the assignment lol

In [55]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import librosa
import resampy
import os
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define a function to extract features from the given filename. You will have to use librosa library for this. You can vary the features however you like. The preferred features are mfcc, chroma and melspectogram.

In [ ]:
def extract_features(filename):
    audio, sr = librosa.load(filename, res_type='kaiser_fast')
    feature_list = []

    mfcc_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc_features.T, axis=0)
    feature_list.append(mfcc_mean)

    chroma_stft = np.abs(librosa.stft(audio))
    chroma_features = librosa.feature.chroma_stft(S=chroma_stft, sr=sr)
    chroma_mean = np.mean(chroma_features.T, axis=0)
    feature_list.append(chroma_mean)

    mel_features = librosa.feature.melspectrogram(y=audio, sr=sr)
    mel_mean = np.mean(mel_features.T, axis=0)
    feature_list.append(mel_mean)

    return feature_list[0],feature_list[1],feature_list[2]

Now, load the data from the database.

You can find the database from https://zenodo.org/records/1188976#.XsAXemgzaUk

You need to only download the audio files and not the video files. Also, read up on the naming convention of the files from the webpage.

In [ ]:
def load_data(test_size):
    X = []
    Y = []

    for i in range(1,25):
        I = "{:02}".format(i)
        data_path = f'/content/drive/My Drive/Audio_Speech_Actors/Actor_{I}'

        for file in (os.listdir(path=f'/content/drive/My Drive/Audio_Speech_Actors/Actor_{I}')):
            if file.endswith('.wav'):
                file_path = os.path.join(data_path, file)
                mfccs, chroma, mel = extract_features(file_path)

                feature_row = np.hstack([mfccs, chroma, mel])
                X.append(feature_row)

                label = np.zeros(8)
                label[int(file.split('-')[3])] = 1
                Y.append(label)

    features = np.array(X)
    labels = np.array(Y)

    X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.2, random_state=18)
    return X_train, X_test, Y_train, Y_test

Use the load_data function to load data and split the data using the train_test_split function.

In [59]:
#split the dataset into training and testing data with testing data = 0.2 of total data
X_train, X_test, Y_train, Y_test = load_data(test_size=0.2)

ModuleNotFoundError: No module named 'resampy'

This error is lazily reported, having originally occured in
  File /usr/local/lib/python3.10/dist-packages/librosa/core/audio.py, line 33, in <module>

----> resampy = lazy.load("resampy")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llvmlite
  Running setup.py clean for llvmlite
Failed to build llvmlite
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (llvmlite)


Now you need to define the model for this computation.
For this, we will use simple neural network for this.\
The model will have a hidden layer of 300 nodes and an output layer of [number of emotions] nodes.\
You have to use relu as activation function for the hidden layer and softmax for the output layer.\
Keep the loss categorical-crossentropy, optimizer as adam and accuracy as metric.\
You can use batch-size as 256 and epochs as 300, but it is left upto you :)

In [ ]:
#define the model.
model = model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(Y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'X_train' is not defined

Now everything is easy-peezy. Just fit the model on training data as you usually do, predict for the testing data, print the accuracy. Hehe

In [ ]:
#fit the model on training data with validation split of 0.2
history = model.fit(X_train, Y_train, epochs=300, batch_size=256, validation_data=(X_test, Y_test))

In [ ]:
#predict for testing data and printout the accuracy
loss, accuracy = model.evaluate(X_test, Y_test)
print("Accuracy of the model taken  ->  ", accuracy)

Now comes the learning part :)\
Here's a new thing. Print the classification report of your predictions.

In [ ]:
#print the classification report for the model
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

class_names = ['Neutral','Calm','Happy','Sad','Angry','Fearful','Disgust','Surprised']
print(type(class_names))
report = classification_report(y_test_classes, y_pred_classes, target_names=class_names, labels=range(len(class_names)), zero_division=1)
print(report)

Just for fun. You can explore on callbacks used in model.fit().\
For resources, you can go to https://keras.io/api/callbacks/ and explore on your own. Or just chatgpt :)
Now try the model using EarlyStopping callback, LearningRateScheduler callback and ModelCheckpoint callback.

In [ ]:
#Have fun ..... you just completed the project

The timeline for this is 2 weeks. In the last two weeks, we will be exploring further concepts in AIML or doing some fun activities in the field. Look forward to it :)